In [3]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import dill
import gzip
import logging

sys.path.append(os.path.abspath('../../surmise/emulationmethods'))
from AKSGP import Emulator

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        # logging.FileHandler('emulator_train.log', mode='w'),  # Log to file
        logging.StreamHandler()  # Log to console
    ]
)
logger = logging.getLogger(__name__)

# Mock data

In [19]:
from sklearn.datasets import make_friedman1

X, y = make_friedman1(n_samples=50, noise=0.0, random_state=0)
print(X.shape)
print(y.shape)


(50, 10)
(50,)


# Train and predict

### AKSGP

In [20]:
emu = Emulator(X, Y_mean=y, Y_std=None)
emu.fit(kernel='RBF', nrestarts=5, seed=13)

INFO:AKSGP:Training GPs with RBF kernel...

INFO:AKSGP:  Standardizing input space...
INFO:AKSGP:  Standardizing data...
/Users/jaiswal/miniconda3/envs/jssims_new/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 5 of parameter k2__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/jaiswal/miniconda3/envs/jssims_new/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 6 of parameter k2__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/jaiswal/miniconda3/envs/jssims_new/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:455: ConvergenceWarning: The optimal value found for dimension 7 of parameter k2__length_scale is close to

In [21]:
Xtest, ytest = make_friedman1(n_samples=10, noise=0.0, random_state=0)
means, std_devs = emu.predict(Xtest)
print("% error:\n", (1.0 - means/ytest.reshape(-1, 1))*100)

% error:
 [[ 2.25733882e-06]
 [ 7.93735822e-07]
 [-3.91089054e-05]
 [ 9.73354140e-06]
 [ 7.39794290e-05]
 [ 1.28839495e-08]
 [-1.69194343e-05]
 [-3.16658411e-07]
 [ 5.57224604e-06]
 [-2.45258599e-05]]


### PCGP

### PCGPR

### PCSK